In [2]:
import urllib
from sqlalchemy import create_engine
import pandas as pd
from datetime import datetime

server='DESKTOP-HJVSCEN\MSSQLSERVER1'
database='Python ETL'
username='sa'
password='Ka@12345678'


ConnectionString = f"""
    DRIVER={{ODBC Driver 18 for SQL Server}};
    SERVER={server};
    DATABASE={database};
    UID={username};
    PWD={password};
    TrustServerCertificate=yes;
"""
# URL-encode the connection string for SQLAlchemy
params=urllib.parse.quote_plus(ConnectionString)

engine=create_engine(f"mssql+pyodbc:///?odbc_connect={params}")

In [3]:
existing_df=pd.read_sql_table('customers_base',con=engine)
existing_df

,customerid,name,city,email,lastupdated
0,101,Tanuj,Hyderabad,rangatanuj@gmail.com,2025-01-20
1,102,Meenu,Hyderabad,meenu@gmail.com,2025-02-22
2,103,John,Pune,john@gmail.com,2025-03-24
3,104,Smrithi,Mumbai,smrithi@gmail.com,2025-04-26
4,105,Chiru,Banglore,chiru@gmail.com,2025-05-28
5,107,Ravi,Delhi,ravi@gmail.com,2025-06-20
6,108,Jack,Delhi,jack@gmail.com,2025-06-20
7,106,Jaaaanu,Delhi,jaaanu@gmail.com,2025-06-24


In [20]:
incoming_df = pd.DataFrame([
    {"customerid": 102, "name": "Meenu", "city": "Hyderabad", "email": "meenu@gmail.com", "lastupdated": datetime(2025, 2, 22)},     # No change
    {"customerid": 104, "name": "Smrithi", "city": "Chennai", "email": "smrithi@gmail.com", "lastupdated": datetime(2025, 6, 23)},   # Changed city,lastupdated
    {"customerid": 101, "name": "Tanuj", "city": "Bangalore", "email": "tanuj.new@gmail.com", "lastupdated": datetime(2025, 6, 23)},  # Changed city and email,lastupdated
    {"customerid": 102, "name": "Meenu", "city": "Hyderabad", "email": "meenu@gmail.com", "lastupdated": datetime(2025, 2, 22)},     # No change
    {"customerid": 106, "name": "Jaaaanu", "city": "Delhi", "email": "jaaanu@gmail.com", "lastupdated": datetime(2025, 6, 24)}            # New record
])
incoming_df

,customerid,name,city,email,lastupdated
0,102,Meenu,Hyderabad,meenu@gmail.com,2025-02-22
1,104,Smrithi,Chennai,smrithi@gmail.com,2025-06-23
2,101,Tanuj,Bangalore,tanuj.new@gmail.com,2025-06-23
3,102,Meenu,Hyderabad,meenu@gmail.com,2025-02-22
4,106,Jaaaanu,Delhi,jaaanu@gmail.com,2025-06-24


In [21]:
# Step 0: Remove duplicates by keeping the most recent record (latest lastupdated)
incoming_df = incoming_df.sort_values('lastupdated').drop_duplicates(subset='customerid', keep='last')


In [22]:
incoming_df.set_index('customerid', inplace=True)
existing_df.set_index('customerid', inplace=True)

In [13]:
incoming_df=incoming_df[existing_df.columns]
incoming_df

,name,city,email,lastupdated
customerid,,,,
102,Meenu,Hyderabad,meenu@gmail.com,2025-02-22
104,Smrithi,Chennai,smrithi@gmail.com,2025-06-23
101,Tanuj,Bangalore,tanuj.new@gmail.com,2025-06-23
107,Jaaaanu,Delhi,jaaanu@gmail.com,2025-06-24


In [23]:
common_ids=incoming_df.index.intersection(existing_df.index)
new_ids=incoming_df.index.difference(existing_df.index)
print(common_ids)
print(new_ids)

Index([102, 104, 101], dtype='int64', name='customerid')
Index([106], dtype='int64', name='customerid')


In [24]:
changed_rows = incoming_df.loc[common_ids] != existing_df.loc[common_ids]
changed_rows

,name,city,email,lastupdated
customerid,,,,
102,False,False,False,False
104,False,True,False,True
101,False,True,True,True


In [25]:
updated_customers = changed_rows.any(axis=1)
updated_customers


customerid
102    False
104     True
101     True
dtype: bool

In [26]:
from sqlalchemy import update,MetaData,column,table

conn=engine.connect()
metadata=MetaData()
customer_table=table('customers_base',
        column('customerid'),
        column('name'),
        column('city'),
        column('email'),
        column('lastupdated')
        )

In [27]:
for cust_id in updated_customers[updated_customers==True].index:
    changed_cols=changed_rows.loc[cust_id]
    # print("The changed columns for row are:",changed_cols)
    # print(changed_cols.index)
    updated_values={}
    for col in changed_cols.index:
        # print(col)
        if(changed_cols[col]):
            # print(cust_id,col)
            new_val=incoming_df.loc[cust_id,col]
            updated_values[col]=new_val
            # print(updated_values)
    # print(cust_id,updated_values) 
    updated_values['lastupdated'] = datetime.now()
    stmt=(
        update(customer_table)
        .where(customer_table.c.customerid==cust_id )
        .values(**updated_values)
    )
    conn.execute(stmt)


In [28]:
only_new_df=incoming_df.loc[new_ids]
if not only_new_df.empty:
    only_new_df.reset_index(inplace=True)
    only_new_df.to_sql('customers_base',con=engine,index=False,if_exists='append')


<bound method Connection.commit of <sqlalchemy.engine.base.Connection object at 0x000002614AD97790>>

In [ ]:
existing_df=pd.read_sql_table('customers_base',con=engine)
existing_df